### The report of project 5
------
Vehicle Detection Project

The goals / steps of this project are the following:

    1.Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
    2.Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.

    3.Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
    4.Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
    5.Estimate a bounding box for vehicles detected.
--------

[//]: # (Image References)

[image1]: ./output_images/image1.png "first"
[image2]: ./output_images/image2.png "two"
[image3]: ./output_images/image3.png "3"
[image4]: ./output_images/image4.png "4"
[image5]: ./output_images/image5.png "5"
[image6]: ./output_images/image6.png "6"
[image7]: ./output_images/image7.png " 8"
[image8]: ./output_images/image8.png "7"
[video1]: ./project_video.mp4 "Video"

### Histogram of oriendted gradients(HOG)
    1.Explain how (and identify where in your code) you extracted HOG features from the training images.

The code for this step is contained in the 1st to 4th cell in the P5.ipynb file. The 1st cell I import the needed packages, read the car and not car datase. Here is an example of one of each the car and non car classes:

![image1](./output_images/image1.png)

    2.Explain how you settled on your final choice of HOG parameters.
The HOG parameters orient=9, pix_per_cell=8, cell_per_block=2 are easy to do the descision. It is difficulty to choose which color space and channel. Actually I test the HOG output features in the SVM, compare the score and speed.

I define a function named get_hog(image) to get the hog feature and hog image. The parameters are orientations, pixels_per_cell, cells_per_block and RGB color channel. Here are example using RGB and gray color space and HOG parameters of orientations=8, pixels_per_cell=(8,8), cells_per_block=(2,2):
![image2](./output_images/image2.png)

The value means, 983 stands for orientations=9, pixels_per_cell=(8,8), cells_per_block=(3,3):
![image3](./output_images/image3.png)

    3.Describe how (and identify where in your code) you trained a classifier using your selected HOG features (and color features if you used them).
The code is in the 6th cell of P5.ipynb. I define the svm with sklearn by using clf = SVC(kernel='linear'), the input image data is GTI vehicle image database, I use function single_img_features(image) to get the feature data. From sklearn.preprocessing import StandardScaler to make the features scaled to zero mean and unit variance, use train_test_split function to split the training data and test data. 
This table I compare the score and time cost to train the SVM:

head| spatial_feat & hist_feat& hog_feat_tine| score | time
---|---|---|---|
YUV|123|0.98|104s|
HLS|123|0.99|89s|
LUV|123|0.982|139s|
HSV|123|0.99|88s|

But when I get a good classifier(high score), the sliding window search is hard to work. So I just set the SVM, with RGB color channel with hog, histogram and spatial binning color.

-----
### Sliding Window Search
    1.Describe how (and identify where in your code) you implemented a sliding window search. How did you decide what scales to search and how much to overlap windows?
The code is defined in the 8th cell. Function slide_window(), function search_windows() and pipeline_to_window().

slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], xy_window=(64, 64), xy_overlap=(0.5, 0.5)), The img is the input shape of 780 x 1280 x 3, x_start_stop and y_start_stop means the detection region of the image, the xy_window means the bounding box's size, and the xy_overlap means the moving step of bounding box, so I x_start_stop from 100 to 1280 with y_start_stop from 380 to 700, set The output is the bboxes in the region.


search_windows(img, windows, clf, scaler), this function is used to classify the slide_window's output. Using the bboxes to take the small part of original image and then throw them into the SVM classifier.

pipeline_to_window(), this function is used to make a pipeline, and draw the vehicle boxes.

![image4](./output_images/image_sliding_window.png)

    2.Show some examples of test images to demonstrate how your pipeline is working. What did you do to optimize the performance of your classifier?
    
Ultimately I searched on two scales using RGB color channel, HOG features plus spatially binned color and histograms of color in the feature vector, which provided a nice result. Here are some example images:

![image5](./output_images/image5.png)

--------

### Video Implementation
    1. Provide a link to your final video output. Your pipeline should perform reasonably well on the entire project video (somewhat wobbly or unstable bounding boxes are ok as long as you are identifying the vehicles most of the time with minimal false positives.)
Here's a link to my video result:

[video](./project_video_output_final.mp4)

    2. Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.

The code is from 16th to the last cell in P5.ipynb. The pipeline_hot_map(image) is used to produce the bboxes and throw them into the SVM. For the boxes distinguished as a car, I created a heatmap and then thresholded the map to identify vehicle positions. By using heat[heat <= threshold] = 0, regrad the heat map as a mask, and only keep the hot area. Because of I set the xy_window as 64, and overlap as 0.5, I just make the threshold as 1 to limit the false positives.

Here is one frames and its corresponding heatmap:
![image6](./output_images/image6.png)

Here is the output of scipy.ndimage.measurements.label() on the integrated heatmap from the frame:
![image7](./output_images/image7.png)

Here the resulting bounding boxes are drawn onto the last frame in the series:
![image8](./output_images/image8.png)

-----
Discussion includes some consideration of problems/issues faced, what could be improved about their algorithm/pipeline, and what hypothetical cases would cause their pipeline to fail.

The boxes in the distance can smaller, and the boxes nearby should be bigger. Different ratio of height and weight may improve the perfomrance.
Besides that, I think the deep learning methods can get a better job, like SSD, YoLo or maskRCNN, and I think it is better to use the GPU to do all the training and inferencing steps, the CPU is too slow for the project.